In [ ]:
#!pip install numpy==1.26.4
#!pip install highway-env
#!pip install git+https://github.com/DLR-RM/stable-baselines3

In [2]:
import random, time, sys

import pygame
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
from gymnasium  import spaces

pygame 2.6.0 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class GridLineEnv(gym.Env):
    """
    Ambiente personalizado onde o agente deve percorrer uma linha em uma grade NxN.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, grid_size=10, line_length=10, line_reward=1.0, goal_reward=10.0, gray_penalty=-2.0, walk_penalty=-1.0):
        super(GridLineEnv, self).__init__()

        self.grid_size = grid_size
        self.line_length = line_length
        self.line_reward = line_reward
        self.goal_reward = goal_reward
        self.gray_penalty = gray_penalty

        # Definir espaço de ação: 8 direções
        self.action_space = spaces.Discrete(8)
        # Definir espaço de observação: posição do agente na grade
        self.observation_space = spaces.MultiDiscrete([grid_size, grid_size])

        # Configuração da linha
        self.line_positions = []
        for i in range(line_length):
            self.line_positions.append((i+1, grid_size // 2))  # Linha horizontal no meio da grade

        self.start_position = self.line_positions[0]
        self.end_position = self.line_positions[-1]
        self.state = self.start_position

        # Recompensas e penalidades
        self.rewards = {
            'line': line_reward,
            'goal': goal_reward,
            'gray': gray_penalty,
            'walk': walk_penalty
        }

        # Configuração do pygame
        self.window_size = 400  # Tamanho da janela
        self.cell_size = self.window_size // self.grid_size
        self.screen = None
        self.clock = None

    def reset(self):
        self.state = self.start_position
        return np.array(self.state)

    def step(self, action):
        done = False
        info = {}

        # Movimentos possíveis
        moves = {
            0: (-1,  0),  # Esquerda
            1: (-1, -1),  # Diagonal superior esquerda
            2: ( 0, -1),  # Cima
            3: ( 1, -1),  # Diagonal superior direita
            4: ( 1,  0),  # Direita
            5: ( 1,  1),  # Diagonal inferior direita
            6: ( 0,  1),  # Baixo
            7: (-1,  1),  # Diagonal inferior esquerda
        }

        move = moves.get(action, (0, 0))
        new_state = (self.state[0] + move[0], self.state[1] + move[1])

        # Verificar limites da grade
        if 0 <= new_state[0] < self.grid_size and 0 <= new_state[1] < self.grid_size:
            self.state = new_state
        else:
            # Se bater na parede, permanece no mesmo lugar
            pass

        # Calcular recompensa
        if self.state == self.end_position:
            reward = self.rewards['goal']
            done = True
        elif self.state in self.line_positions:
            reward = self.rewards['line'] + self.rewards['walk']
        else:
            reward = self.rewards['gray'] + self.rewards['walk']

        return np.array(self.state), reward, done, info

    def render(self, mode='human'):
        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.window_size, self.window_size))
            pygame.display.set_caption("Grid Line Environment")
            self.clock = pygame.time.Clock()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # Desenhar fundo
        self.screen.fill((128, 128, 128))  # Cinza

        # Desenhar linha
        for pos in self.line_positions:
            x, y = pos
            rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
            pygame.draw.rect(self.screen, (0, 0, 0), rect)  # Preto

        # Desenhar objetivo
        x, y = self.end_position
        rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
        pygame.draw.rect(self.screen, (255, 255, 0), rect)  # Amarelo

        # Desenhar agente
        x, y = self.state
        rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
        pygame.draw.rect(self.screen, (255, 0, 0), rect)  # Vermelho

        pygame.display.flip()
        self.clock.tick(5)  # Controla a velocidade da renderização

    def close(self):
        if self.screen is not None:
            pygame.quit()
            self.screen = None


In [4]:
class MonteCarloAgent:
    """
    Agente que utiliza o método de Controle Monte Carlo com Exploração-First.
    """
    def __init__(self, env, epsilon=0.1):
        self.env = env
        self.states = [(x, y) for x in range(env.grid_size) for y in range(env.grid_size)]
        self.actions = list(range(env.action_space.n))
        self.gamma = 0.9  # Fator de desconto
        self.epsilon = epsilon  # Parâmetro para ε-greedy

        # Inicializa Q(s, a) e as listas de retornos
        self.Q = {}
        self.returns = {}
        for state in self.states:
            for action in self.actions:
                self.Q[(state, action)] = 0.0
                self.returns[(state, action)] = []

        # Política inicial (aleatória)
        self.policy = {}
        for state in self.states:
            self.policy[state] = np.ones(env.action_space.n) / env.action_space.n

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            # Escolhe uma ação aleatória (exploração)
            return np.random.choice(self.actions)
        else:
            # Escolhe a melhor ação com base em Q(s, a) (exploração)
            q_values = [self.Q.get((state, a), 0.0) for a in self.actions]
            max_q = np.max(q_values)
            # Pode haver várias ações com o mesmo valor máximo
            best_actions = [a for a, q in zip(self.actions, q_values) if q == max_q]
            return np.random.choice(best_actions)

    def generate_episode(self):
        episode = []
        state = tuple(self.env.reset())
        done = False

        while not done:
            action = self.choose_action(state)
            next_state, reward, done, _ = self.env.step(action)
            next_state = tuple(next_state)
            episode.append((state, action, reward))
            state = next_state

        return episode

    def update_policy(self):
        for state in self.states:
            q_values = [self.Q.get((state, a), 0.0) for a in self.actions]
            max_q = np.max(q_values)
            best_actions = [a for a, q in zip(self.actions, q_values) if q == max_q]
            # Política ε-greedy
            self.policy[state] = np.ones(len(self.actions)) * self.epsilon / len(self.actions)
            for a in best_actions:
                self.policy[state][a] += (1.0 - self.epsilon) / len(best_actions)

    def monte_carlo_control(self, episodes=1000):
        for _ in range(episodes):
            episode = self.generate_episode()
            G = 0
            visited_state_action_pairs = set()
            for state, action, reward in reversed(episode):
                G = reward + self.gamma * G
                if (state, action) not in visited_state_action_pairs:
                    self.returns[(state, action)].append(G)
                    self.Q[(state, action)] = np.mean(self.returns[(state, action)])
                    visited_state_action_pairs.add((state, action))
            # Atualiza a política após cada episódio
            self.update_policy()

    def train(self, episodes=1000):
        self.monte_carlo_control(episodes)


In [6]:
# Código para executar o ambiente e o agente
if __name__ == "__main__":
    # Configurações
    grid_size = 30
    line_length = 15
    line_reward = .5
    goal_reward = 10.0
    gray_penalty = -.5
    walk_penalty = -.5
    episodes = 100  # Número de episódios para treinamento

    print('creating environment...')
    env = GridLineEnv(grid_size=grid_size, line_length=line_length,
                      line_reward=line_reward, goal_reward=goal_reward,
                      gray_penalty=gray_penalty, walk_penalty=walk_penalty)

    agent = MonteCarloAgent(env)
    print('training...')
    agent.train(episodes=episodes)

    # Executar episódios com renderização
    for episode in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            #print('rendering...')
            env.render()
            state_tuple = tuple(state)
            action_probs = agent.policy[state_tuple]
            action = np.random.choice(agent.actions, p=action_probs)
            state, reward, done, _ = env.step(action)
            total_reward += reward

        print(f"Episódio {episode + 1}: Recompensa total = {total_reward}")
        time.sleep(1)

    env.close()


creating environment...
training...
Episódio 1: Recompensa total = -14.0
Episódio 2: Recompensa total = -23.0
Episódio 3: Recompensa total = -12.0
Episódio 4: Recompensa total = -18.0
Episódio 5: Recompensa total = -9.0
Episódio 6: Recompensa total = -1.0
Episódio 7: Recompensa total = -19.0
Episódio 8: Recompensa total = -2.0
Episódio 9: Recompensa total = 2.0
Episódio 10: Recompensa total = -18.0
Episódio 11: Recompensa total = -10.0
Episódio 12: Recompensa total = -9.0
Episódio 13: Recompensa total = -15.0
Episódio 14: Recompensa total = -11.0
Episódio 15: Recompensa total = -1.0
Episódio 16: Recompensa total = -15.0
Episódio 17: Recompensa total = -38.0
Episódio 18: Recompensa total = -38.0
Episódio 19: Recompensa total = -21.0
Episódio 20: Recompensa total = -9.0
Episódio 21: Recompensa total = -2.0
Episódio 22: Recompensa total = -10.0
Episódio 23: Recompensa total = -31.0
Episódio 24: Recompensa total = -4.0
Episódio 25: Recompensa total = -9.0
Episódio 26: Recompensa total = -1

SystemExit: 